# 14. 데이터베이스
SQLite DB를 사용해서 강의 진행

## pysqlite 모듈
sqlite db를 연결하기 위한 모듈

### 데이터베이스 연결
물리적인 DB 파일이 없으면 생성되며, 파일이 이미 존재하면 그 DB 파일을 사용한다.

In [2]:
import sqlite3
con = sqlite3.connect("test.db")  # 물리적인 파일 생성
con = sqlite3.connect(":memory:") # 메모리에 생성 

### SQL 문 실행

In [3]:
import sqlite3
con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table PhoneBook(Name text, PhoneNum text);")

In [5]:
cur.execute("insert into PhoneBook values ('Derick', '010-1234-5678');")

### 레코드 조회
execute() 매소를 사용해서 select절 수행가능.

In [7]:
cur.execute("select * from PhoneBook;")
for row in cur:
    print(row)

('Derick', '010-1234-5678')
('Derick', '010-1234-5678')


In [8]:
name = "sungjin"
phoneNumver = "010-123-4454"
cur.execute("insert into PhoneBook values (?, ?);", (name, phoneNumver))

cur.execute("insert into PhoneBook values (:inputName, :inputPhoneNumber);", {"inputPhoneNumber": phoneNumver, "inputName": name })

In [9]:
cur.execute("select * from PhoneBook;")
for row in cur:
    print(row)

('Derick', '010-1234-5678')
('Derick', '010-1234-5678')
('sungjin', '010-123-4454')
('sungjin', '010-123-4454')


In [10]:
datalist = (('tom', '010-4530-1123'), ('john', '010-8920-1234'))
cur.executemany("insert into PhoneBook values (?, ?);", datalist)

In [11]:
cur.execute("select * from PhoneBook;")
for row in cur:
    print(row)

('Derick', '010-1234-5678')
('Derick', '010-1234-5678')
('sungjin', '010-123-4454')
('sungjin', '010-123-4454')
('tom', '010-4530-1123')
('john', '010-8920-1234')


In [12]:
def dataGenerator():
    datalist = {('madmax', '010-2304-1233'), ('marry', '053-2320-1111')}
    for item in datalist:
        yield item
        
cur.executemany("insert into PhoneBook values (?, ?);", dataGenerator())

In [20]:
cur.execute("select * from PhoneBook;")

cur.fetchone()
cur.fetchone()
cur.fetchone()

cur.fetchmany(2) # list에다 tuple을 담아서 리턴

cur.execute("select * from PhoneBook;")
cur.fetchall()

[('Derick', '010-1234-5678'),
 ('Derick', '010-1234-5678'),
 ('sungjin', '010-123-4454'),
 ('sungjin', '010-123-4454'),
 ('tom', '010-4530-1123'),
 ('john', '010-8920-1234'),
 ('madmax', '010-2304-1233'),
 ('marry', '053-2320-1111')]

##  트랜잭션 처리
여러 개의 SQL 구문을 하나의 실행단위로 묶어서 처리할 수 있다. 아래의 구문을 실행하고 커밋을 하지 않으면 데이터가 저장되지 않는다.
작업한 내용을 commit을 하면 반영된다.

### 자료형
SQLite3은 NULL, INTEGER, REAL, TEXT, BLOB만을 기본 자료형으로 제공한다.

| SQLite3 type  | Python type   |
| ------------- | ------------- |
|Null|None|
|Integer|int|
|REAL|float|
|TEXT|str, bytes|
|BLOB|buffer|

In [2]:
from os.path import *
import sqlite3

con = sqlite3.connect(abspath("test.db"))
cur = con.cursor()
cur.execute("create table PhoneBook(Name text, PhoneNum text);")
cur.execute("insert into PhoneBook values ('Derick', '010-1234-5678');")
cur.execute("select * from PhoneBook;")
print(cur.fetchall())

OperationalError: table PhoneBook already exists

In [1]:
# 위에서 commit을 하지 않았기 때문에, 종료를 하고 재시작해서 select를 날리면 데이터가 없는 것을 확인가능하다.
from os.path import *
import sqlite3

con = sqlite3.connect(abspath("test.db"))
cur = con.cursor()
cur.execute("select * from PhoneBook;")
print(cur.fetchall())

[]


In [1]:
from os.path import *
abspath("test2.db")

'/Users/bookboogie/p/git/start-python/14. 데이터베이스/test2.db'

In [2]:
from os.path import *
import sqlite3

con = sqlite3.connect(abspath("test2.db"))
cur = con.cursor()
cur.execute("create table PhoneBook(Name text, PhoneNum text);")
cur.execute("insert into PhoneBook values ('Derick', '010-1234-5678');")
cur.execute("select * from PhoneBook;")
print(cur.fetchall())

con.commit()

[('Derick', '010-1234-5678')]


In [3]:
# 위에서 commit을 하지 않았기 때문에, 종료를 하고 재시작해서 select를 날리면 데이터가 없는 것을 확인가능하다.
from os.path import *
import sqlite3

con = sqlite3.connect(abspath("test2.db"))
cur = con.cursor()
cur.execute("select * from PhoneBook;")
print(cur.fetchall())

[('Derick', '010-1234-5678')]


### 데이터베이스 백업
현재 상태를 SQL 구문으로 추출해야 할때가 있다.
이런 경우 Connection.iteratordump()를 이용해서 추출 가능하다. 호출 결과는 SQL 구문을 iterator 형태로 반환

In [4]:
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table PhoneBook(Name text, PhoneNum text);")
cur.execute("insert into PhoneBook values ('Derick', '010-1234-5678');")
cur.execute("insert into PhoneBook values ('John', '010-156-9877');")
cur.execute("insert into PhoneBook values ('Marry', '010-8977-1111');")

for item in con.iterdump():
    print(item)

BEGIN TRANSACTION;
CREATE TABLE PhoneBook(Name text, PhoneNum text);
INSERT INTO "PhoneBook" VALUES('Derick','010-1234-5678');
INSERT INTO "PhoneBook" VALUES('John','010-156-9877');
INSERT INTO "PhoneBook" VALUES('Marry','010-8977-1111');
COMMIT;


In [6]:
from os.path import *
import sqlite3

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table PhoneBook(Name text, PhoneNum text);")
cur.execute("insert into PhoneBook values ('Derick', '010-1234-5678');")
cur.execute("insert into PhoneBook values ('John', '010-156-9877');")
cur.execute("insert into PhoneBook values ('Marry', '010-8977-1111');")

#파일로 저장
with open(abspath("dump.sql"), "w") as f:
    for sql in con.iterdump():
        f.write("{0}\n".format(sql))

In [7]:
import sqlite3
con = sqlite3.connect(":memory:")
with open(abspath("dump.sql")) as f:
    SQLScript = f.read()
    
cur = con.cursor()
cur.executescript(SQLScript) #dump로 생성한 script를 실행해서 복구 가능하다.

In [8]:
cur.execute("select * from PhoneBook;")
cur.fetchall()

[('Derick', '010-1234-5678'),
 ('John', '010-156-9877'),
 ('Marry', '010-8977-1111')]

In [11]:
cur.execute("insert into PhoneBook values ('Admin', '02-111-9999');")

In [12]:
for d in con.iterdump():
    print(d)

BEGIN TRANSACTION;
CREATE TABLE PhoneBook(Name text, PhoneNum text);
INSERT INTO "PhoneBook" VALUES('Derick','010-1234-5678');
INSERT INTO "PhoneBook" VALUES('John','010-156-9877');
INSERT INTO "PhoneBook" VALUES('Marry','010-8977-1111');
INSERT INTO "PhoneBook" VALUES('Admin','02-111-9999');
COMMIT;
